In [1]:
%pip install -qU langchain-openai langchain langgraph langchain_community langgraph-supervisor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 887.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 9.9 MB/s eta 0:00:00


In [2]:
# !pip install -qU  googletrans==4.0.0-rc1
!pip install -qU googletrans


In [3]:
!pip install  -qU langchain-openai openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.3/730.3 kB 8.6 MB/s eta 0:00:00


In [4]:
from pydantic import BaseModel

from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
)

key = userdata.get('OPENAI_API_KEY')

In [5]:
from langchain_openai import ChatOpenAI

In [6]:
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

In [19]:
from langchain.tools import tool
from googletrans import Translator

@tool
async def localize_to_nepali(content: str) -> str:
    """
    Translates/adapts the given English content to Nepali.

    Args:
        content (str): The English text content to be localized.

    Returns:
        str: The translated content in Nepali.
    """
    translator = Translator()
    try:
        translated = await translator.translate(content, src='en', dest='ne')
        return translated.text
    except Exception as e:
        return f"Translation failed due to: {str(e)}"


In [17]:
result = await localize_to_nepali("Hello! Welcome to our service. How can I help you today?")
print(result)


नमस्ते! हाम्रो सेवामा स्वागत छ। आज म तपाईंलाई कसरी मद्दत गर्न सक्छु?


In [18]:
result = await localize_to_nepali("Hello! How are you?")
print(result)


नमस्ते! तिमीलाई कस्तो छ?


Works

# Tools

In [30]:
import asyncio

@tool
def localize_to_nepali(content: str) -> str:
    """
    Translates English text into Nepali using an async translator wrapped in sync.
    """   # Hypothetical example
    async def translate_async():
        translator = Translator()
        translated = await translator.translate(content, src='en', dest='ne')
        return translated.text

    try:
        return asyncio.run(translate_async())
    except Exception as e:
        return f"Translation failed due to: {str(e)}"


# Agent

In [31]:
localization_assistant = create_react_agent(
    model=model,
    tools=[localize_to_nepali],
    prompt=(
    "You are a helpful Localization Assistant. "
    "Your job is to translate and adapt English content into the Nepali language, "
    "preserving the original meaning, tone, and cultural context. "
    "Always use the `localize_to_nepali` tool to translate any user input into Nepali. "
    "Do not respond on your own unless explicitly instructed."
),
    name="localization_assistant"
)


# Agent Invoke

In [32]:
response = localization_assistant.invoke(
    {
        "messages": [
             HumanMessage(content="Translate to Nepali: Hello. How are you doing?")

            ]
    }, config={"recursion_limit": 50}
    )
response["messages"]

[HumanMessage(content='Translate to Nepali: Hello. How are you doing?', additional_kwargs={}, response_metadata={}, id='13845be3-815d-4abc-8432-660be6609380'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hjuyHESpsHnodhYKqfoHivA1', 'function': {'arguments': '{"content":"Hello. How are you doing?"}', 'name': 'localize_to_nepali'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 132, 'total_tokens': 156, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bicd2PyAR6oIy6w7neQF2nTq0Je2w', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='localization_assistant', id='run--1838bbc3-0f70-4aec-adff-947cf44b9860-0', tool_call

In [33]:
response["messages"][-1].content

'नमस्कार। तिमीलाई कस्तो छ?'

In [34]:
response = localization_assistant.invoke(
    {
        "messages": [
             HumanMessage(content="Translate to Nepali: Today is  a nice day for picnic.")

            ]
    }, config={"recursion_limit": 50}
    )
response["messages"]

[HumanMessage(content='Translate to Nepali: Today is  a nice day for picnic.', additional_kwargs={}, response_metadata={}, id='18860915-7ca8-46c8-a319-bcf408490a01'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_o6NUXNfLKUD9lIflLqFPvzBz', 'function': {'arguments': '{"content":"Today is a nice day for picnic."}', 'name': 'localize_to_nepali'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 134, 'total_tokens': 159, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BicdvR1QyV44Qlqwwd3j3rQlMXLpC', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='localization_assistant', id='run--c445f287-7a8b-4007-b7f6-8999d893cef1-

In [35]:
response["messages"][-1].content

'आज पिकनिकको लागि राम्रो दिन हो।'

In [38]:
response = localization_assistant.invoke(
    {
        "messages": [
             HumanMessage(content="Today is not sunny day.")

            ]
    }, config={"recursion_limit": 50}
    )
response["messages"]

[HumanMessage(content='Today is not sunny day.', additional_kwargs={}, response_metadata={}, id='210e26fb-d858-4257-8b92-424f79ef0be1'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5Z3eWDQBGzai5pKLW1iIaMOh', 'function': {'arguments': '{"content":"Today is not sunny day."}', 'name': 'localize_to_nepali'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 126, 'total_tokens': 149, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Biclpu5AMuRYVJzaRhjlGf1Vvb1fj', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='localization_assistant', id='run--8baaf9a0-2df3-40a8-a3c8-e2c65d5c62f2-0', tool_calls=[{'name': 'localize_to_

In [39]:
response["messages"][-1].content

'आज घमाइलो दिन होइन।'

# Class Local

# Localization Tool. PY

In [ ]:
import asyncio
from langchain.tools import tool
from typing import List
import os

@tool
def localize_to_nepali(content: str) -> str:
    """
    Translates English text into Nepali using an async translator wrapped in sync.
    """
    async def translate_async():
        translator = Translator()
        translated = await translator.translate(content, src='en', dest='ne')
        return translated.text

    try:
        return asyncio.run(translate_async())
    except Exception as e:
        return f"Translation failed due to: {str(e)}"


# Run_localization agent.py

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from agents.localization_agent import LocalizationAgent
def main():
    agent = LocalizationAgent()
    query = "Hello. How are you doing?"
    result = agent.run(query)
    print(result)

if __name__ == "__main__":
    main()

# Localization Agent py

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from tools.localization_tools import localize_to_nepali
from config.setting import OPENAI_API_KEY

class LocalizationAgent:
    def __init__(self):
        self.openai_api_key = OPENAI_API_KEY
        self.model = ChatOpenAI(api_key=self.openai_api_key, model="gpt-4o-mini", verbose=True)
        self.agent = create_react_agent(
            model=self.model,
            tools=[localize_to_nepali],
            prompt = (
            "You are a helpful Localization Assistant. "
            "Your job is to translate and adapt English content into the Nepali language, "
            "preserving the original meaning, tone, and cultural context. "
            "Always use the `localize_to_nepali` tool to translate any user input into Nepali. "
            "Do not respond on your own unless explicitly instructed."
            ),
            name="localization_assistant"
        )

    def run(self, user_input):
        response = self.agent.invoke({
            "messages": [HumanMessage(content=user_input)]
        }, config={"recursion_limit": 50})
        return response["messages"][-1].content